In [1]:
from bs4 import BeautifulSoup
import requests, re
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [2]:
hospital = "서울대학교병원"
doctor = "송영욱"
query = hospital + '+' + doctor
s_date = "2019.08.01"
e_date = "2019.09.01"
s_from = s_date.replace(",","")
e_to = e_date.replace(".","")
maxpage = 1

In [3]:
def text_app(phara):
    for item in phara:
        text = str(item.find_all(text = True))
        clean = re.sub('[a-zA-Z]','',text)
        clean = re.sub('[^가-힝0-9a-zA-Z\\s]','',clean)
        text_lst.append(clean)
        
def cont_app(contents):
    for cont in contents:
        first = re.sub('<dl>.*?</a> </div> </dd> <dd', '', str(cont)).strip()
        second = re.sub('<ul class = "relation_lst">.*?</dd>','',first).strip()
        third = re.sub('<.+?>','',second).strip()
    return third

In [4]:
page = 1
maxpage_t = (int(maxpage)-1) * 10 + 1

url = "https://search.naver.com/search.naver?where=news&query="\
    + query + "&sort=1&ds=" + s_date + "&de=" + e_date \
    + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to \
    + "%2Ca%3A&start=" + str(page)
url

'https://search.naver.com/search.naver?where=news&query=서울대학교병원+송영욱&sort=1&ds=2019.08.01&de=2019.09.01&nso=so%3Ar%2Cp%3Afrom2019.08.01to20190901%2Ca%3A&start=1'

In [5]:
title_lst = []
link_lst = []
news_lst = []
date_lst = []
cont_lst = []
text_lst = []

while page <= maxpage_t:
    response = requests.get(url)
    html = response.text

    #BeautifulSoup 인자값으로 변환
    soup = BeautifulSoup(html, 'html.parser')
    #<a> 태그에서 제목과 링크 주소 추출
    atags = soup.select('._sp_each_title')
    
#     제목 & 링크주소
    for atag in atags:
        title_lst.append(atag.text)
        link_lst.append(atag['href'])
    
    press_source = soup.select('._sp_each_source')
    for press in press_source:
        news_lst.append(press.text)
        
    date_soup = soup.select('.txt_inline')
    for seg in date_soup:
        state = seg.text
        try:
            pattern = '\d+.(\d+).(\d+).'
            re_comp = re.compile(pattern)
            date = re_comp.search(state).groujp(0)
            date_lst.append(date)
        except AttributeError:
            pattern = '\w* (\d\w*)'
            re_comp = re.compile(pattern)
            date = re_comp.search(state).group(1)
            date_lst.append(date)
            
    contents = soup.select('ul.type01 dl')
    for cont in contents:
        first = re.sub('<dl>.*?</a> </div> </dd> <dd', '', str(cont)).strip()
        second = re.sub('<ul class = "relation_lst">.*?</dd>','',first).strip()
        third = re.sub('<.+?>','',second).strip()
        cont_lst.append(third)
    page+=10
    
for i in link_lst:
    resp = requests.get(i)
    resp.raise_for_status()
    resp.encoding = 'euc-kr'
    htm = resp.content
    sou = BeautifulSoup(htm,'html.parser')
    phara = sou.find_all('div',{'itemprop':'articleBody'})
    if len(phara) == 0:
        phara = sou.find_all('div',{'id':'arl_view_content'})
        if len(phara) == 0:
#             phara = sou.find_all("div",{'class':'bodyarea'})
#             if len(phara) == 0:
            text_lst.append('차단')
        else:
            text_app(phara)

    else:
        text_app(phara)

result = {
    "date":date_lst,
    "title":title_lst,
    "press":news_lst,
    "preview":cont_lst,
    "content":text_lst,
    "link":link_lst
}

df = pd.DataFrame(result)
df

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


,date,title,press,preview,content,link
0,2019,"릴리, ‘올루미언트’ 유용성 확인한 심포지엄 성료",약업신문,">심포지엄은 서울대병원 류마티스내과 송영욱 교수가 좌장을 맡아 주도했으며, 최신 류...",차단,http://www.yakup.com/news/index.html?mode=view...
1,2019,"한국릴리, 경구용 류마티스관절염치료제 ‘올루미언트’ 심포지엄 성료",메디컬투데이,">이번 심포지엄은 서울대병원 류마티스내과 송영욱 교수가 좌장을 맡아 주도했으며, 최...",차단,http://www.mdtoday.co.kr/mdtoday/index.html?no...
2,2019,[사제인사] 서울대교구,가톨릭신문,>병원사목위원회 서울대학교병원 원목부실장) ▲중계동 부주임 박태민(정릉4동 보좌) ...,차단,https://www.catholictimes.org/article/article_...
3,2018,"아모레퍼시픽, 피부세포 항노화 물질 연구 국제학술지 게재",천지일보,>아모레퍼시픽 기술연구원 길인섭 수석연구원과 서울대학교 병원 내분비내과 송영욱 교수...,아모레퍼시픽 기술연구원 미지움 제공 아모레퍼시픽 천지일보김예슬 기자 아모...,http://www.newscj.com/news/articleView.html?id...
4,2018,"아모레퍼시픽, 피부 항노화 물질 연구 국제학술지 게재",이뉴스투데이,>길인섭 아모레퍼시픽 기술연구원 수석연구원과 송영욱 서울대학교 병원 내분비내과 교수...,사진제공아모레퍼시픽 이뉴스투데이 이하영 기자 아모레퍼시픽 기술연구원이 서울대학교 ...,http://www.enewstoday.co.kr/news/articleView.h...
5,2018,"아모레퍼시픽, 피부세포 항노화 물질 연구 국제학술지 게재",소비자가 만드는 신문,>아모레퍼시픽 기술연구원 길인섭 수석연구원과 서울대학교 병원 내분비내과 송영욱 교수...,아모레퍼시픽은 아모레퍼시픽 기술연구원이 서울대학교 병원과 공동 연구를 통해 세포 내...,http://www.consumernews.co.kr/?mod=news&act=ar...
6,2018,"아모레퍼시픽, 피부세포 항노화 물질 연구 국제학술지 게재",비즈트리뷴,>아모레퍼시픽 기술연구원 길인섭 수석연구원과 서울대학교 병원 내분비내과 송영욱 교수...,차단,http://biztribune.co.kr/news/view.php?no=88972
7,2018,"아모레퍼시픽, 업계 최초 피부 세포 노화 억제 효능 입증",데일리한국,>아모레퍼시픽 기술연구원 길인섭 수석연구원과 서울대학교 병원 내분비내과 송영욱 교수...,경기도 용인시에 위치한 아모레퍼시픽 기술연구원 데일리한국 동효정...,http://daily.hankooki.com/lpage/industry/20181...
8,2018,"아모레퍼시픽, 피부세포 항노화 물질 연구 국제학술지 게재",CNB저널,>길인섭 아모레퍼시픽 기술연구원 수석연구원과 서울대학교 병원 내분비내과 송영욱 교수...,차단,http://weekly.cnbnews.com/news/article.html?no...
9,2018,"아모레퍼시픽, 피부세포 항노화 물질 연구 활발",일간투데이,>아모레퍼시픽 기술연구원 길인섭 수석연구원과 서울대학교 병원 내분비내과 송영욱 교수...,0 0 아모레퍼시픽 기술연구원 전경 사진아모레퍼시픽 일간투데...,http://www.dtoday.co.kr/news/articleView.html?...


In [6]:
url1 = 'http://www.mdtoday.co.kr/mdtoday/index.html?no=350564'
test1 = requests.get(url1)
test1.raise_for_status()
test1.encoding = 'euc-kr'
bs1 = BeautifulSoup(test1.text,'html.parser', from_encoding = 'utf-8')

In [7]:
bs1.title.string

'한국릴리, 경구용 류마티스관절염치료제 ‘올루미언트’ 심포지엄 성료'

In [8]:
bs1.title(string = True)

['한국릴리, 경구용 류마티스관절염치료제 ‘올루미언트’ 심포지엄 성료']